In [11]:
import json
import os
import mysql.connector as sql
from settings import *
from gates import *

def load_database_config(path):
    with open(path, 'r') as f:
        config_data = json.load(f)
    
    processed_config = {}
    for key, command in config_data['mysql'].items():
        gate = JSON_GATE(command)
        processed_config[key] = gate.parse_and_execute()
    
    return processed_config

config_path = os.path.join(GAME_DIRECTORY, "configuration settings", "SQL_Link", "config.json")
db_config = load_database_config(config_path)

In [10]:
try:
    with sql.connect(**db_config) as db: # unpacks config dictionary
        with db.cursor() as cursor:
            cursor.execute("CREATE DATABASE IF NOT EXISTS " + TEST_REALM)
            cursor.execute("USE " + TEST_REALM)  # select the database

            # Proceed with table creation and other operations
            cursor.execute("DROP TABLE players")
            cursor.execute("CREATE TABLE IF NOT EXISTS players (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(255))")
            cursor.execute("INSERT INTO players (name) VALUES (%s)", ("player 1",)) 
            db.commit()

            cursor.execute("SELECT * FROM players")
            for row in cursor:
                print(row)
except sql.Error as err:
    print("Something went wrong:", err)


try:
    with sql.connect(**db_config) as db: # unpacks config dictionary
        with db.cursor() as cursor:
            cursor.execute("CREATE DATABASE IF NOT EXISTS " + TEST_REALM)
            cursor.execute("USE " + TEST_REALM)  # select the database

            # Set up the stored procedure
            cursor.execute("""
                CREATE PROCEDURE level_up(IN character_id INT, IN exp_gained INT)
                BEGIN
                    UPDATE characters SET experience = experience + exp_gained WHERE id = character_id;
                    UPDATE characters SET level = level + 1 WHERE id = character_id AND experience >= 1000;
                END
            """)
except sql.Error as err:
    print("Something went wrong:", err)

(1, 'player_name')
Something went wrong: 1304 (42000): PROCEDURE level_up already exists


In [41]:
import json
import mysql.connector
from settings import *

# Load MySQL credentials from configuration file
with open(GAME_DIRECTORY + "\\configuration settings\\SQL_Link\\config.json", 'r') as f:
    config = json.load(f)

try:
    with mysql.connector.connect(**config['mysql']) as db:  # unpacks config dictionary
        with db.cursor() as cursor:
            cursor.execute("USE testdatabase") # select database

            # Create the stored procedure
            cursor.execute("""
                CREATE PROCEDURE level_up(IN character_id INT, IN exp_gained INT)
                BEGIN
                    UPDATE characters SET experience = experience + exp_gained WHERE id = character_id;
                    UPDATE characters SET level = level + 1 WHERE id = character_id AND experience >= level*1000;
                END;
            """)
            db.commit()
except mysql.connector.Error as err:
    print("Something went wrong:", err)

            cursor.execute("CREATE DATABASE IF NOT EXISTS " + TEST_REALM)
            cursor.execute("USE " + TEST_REALM)  # select the database

            # Drop existing tables if needed (for a fresh setup)
            cursor.execute("DROP TABLE IF EXISTS characters")
            cursor.execute("DROP TABLE IF EXISTS players")

            # Create new tables
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS characters (
                    id INT AUTO_INCREMENT PRIMARY KEY, 
                    name VARCHAR(255),
                    experience INT DEFAULT 0,
                    level INT DEFAULT 1
                )
            """)
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS players (
                    id INT AUTO_INCREMENT PRIMARY KEY, 
                    name VARCHAR(255)
                )
            """)
            
            # Insert a sample character
            cursor.execute("INSERT INTO characters (name, experience, level) VALUES (%s, %s, %s)", ("John Doe", 500, 1))
            db.commit()
            
            # Drop the stored procedure if it exists
            cursor.execute("DROP PROCEDURE IF EXISTS level_up")

            # Create the stored procedure
            cursor.execute("""
                CREATE PROCEDURE level_up(IN character_id INT, IN exp_gained INT)
                BEGIN
                    UPDATE characters SET experience = experience + exp_gained WHERE id = character_id;
                    UPDATE characters SET level = level + 1 WHERE id = character_id AND experience >= level*1000;
                END;
            """)
            db.commit()

except mysql.connector.Error as err:
    print("Something went wrong:", err)



In [42]:
# TEST STORED PROCEDURE

try:
    with mysql.connector.connect(**config['mysql']) as db:  # unpacks config dictionary
        with db.cursor() as cursor:
            cursor.execute("USE " + TEST_REALM)  # select the database

            # Call the stored procedure
            cursor.callproc('level_up', [1, 500])  # Assuming `1` is the ID of Rowan Raya Lucaria
            
            # Fetch the updated character data
            cursor.execute("SELECT id, name, experience, level FROM characters WHERE id = 1")
            for row in cursor:
                print(row)  # Outputs the updated character data

except mysql.connector.Error as err:
    print("Something went wrong:", err)


(1, 'John Doe', 1000, 2)
